In [1]:
import numpy as np

In [2]:
X1=np.load("X1_25.npy")
X2=np.load("X2_25.npy")

In [3]:
X1.shape

(27672, 25, 6, 848)

In [4]:
X2.shape

(20929, 25, 6, 848)

In [5]:
X1=np.float32(X1)
X2=np.float32(X2)


In [6]:
from keras.layers import Input, Dense, Reshape, Flatten,Dropout
from keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

input_shape = (25,6,848)
hidden_size1 = 256
hidden_size2 = 128
hidden_size3 = 100
code_size = 64

input_layer = Input(shape=(input_shape))
hidden_layer_0=Flatten()(input_layer)
hidden_layer_1 = Dense(hidden_size1, activation='relu')(hidden_layer_0)
hidden_layer_2 = Dense(hidden_size2, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(hidden_size3, activation='relu')(hidden_layer_2)
bottleneck_layer = Dense(code_size*25, activation='linear')(hidden_layer_3)

# Define the decoder network
hidden_layer_5 = Dense(hidden_size1, activation='relu')(bottleneck_layer)
hidden_layer_6 = Dense(hidden_size2, activation='relu')(hidden_layer_5)
hidden_layer_7 = Dense(hidden_size1, activation='relu')(hidden_layer_6)
hidden_layer_8 = Dense(6*848*25, activation='linear')(hidden_layer_7)
output_layer=Reshape((25,6,848))(hidden_layer_8)

autoencoder = Model(input_layer, output_layer)
opt = Adam(learning_rate=0.001)
autoencoder.compile(optimizer=opt, loss='mse')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')
autoencoder.fit(X1, X1, epochs=10,batch_size=64,
                validation_data=(X2, X2),callbacks=[early_stopping]
              )

Epoch 1/10
433/433 [==============================] - 72s 166ms/step - loss: 0.0104 - val_loss: 0.0115
Epoch 2/10
433/433 [==============================] - 71s 165ms/step - loss: 0.0032 - val_loss: 0.0118
Epoch 3/10
433/433 [==============================] - 72s 167ms/step - loss: 0.0027 - val_loss: 0.0104
Epoch 4/10
433/433 [==============================] - 72s 167ms/step - loss: 0.0026 - val_loss: 0.0105
Epoch 5/10
433/433 [==============================] - 73s 168ms/step - loss: 0.0023 - val_loss: 0.0112
Epoch 6/10
433/433 [==============================] - 73s 169ms/step - loss: 0.0023 - val_loss: 0.0105
Epoch 7/10
433/433 [==============================] - 72s 166ms/step - loss: 0.0023 - val_loss: 0.0101
Epoch 8/10
433/433 [==============================] - 72s 165ms/step - loss: 0.0023 - val_loss: 0.0101
Epoch 9/10
433/433 [==============================] - 72s 167ms/step - loss: 0.0021 - val_loss: 0.0107
Epoch 10/10
433/433 [==============================] - 71s 165ms/step - l

In [7]:
encoder = Model(input_layer, bottleneck_layer)
encoder.compile(optimizer='adam', loss='mse')

In [8]:
X11=encoder.predict(X1)
X22=encoder.predict(X2)

655/655 [==============================] - 8s 12ms/step


In [9]:
decoder = Model(bottleneck_layer, output_layer)
decoder.compile(optimizer='adam', loss='mse')

In [10]:
Y1=decoder.predict(X11)
Y2=decoder.predict(X22)

689/689 [==============================] - 3s 4ms/step


In [18]:
np.save("X11_25",X11)
np.save("X22_25",X22)

In [9]:
encoder.save("encoder-simple_25.h5")

# Cnn encoder

In [10]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D,Dropout
from keras.models import Model

# Define input shape
input_shape = (25,6, 848)

# Define encoder architecture
inputs = Input(shape=input_shape)
x = Conv2D(128, 3, activation='relu', padding='same' )(inputs)
x = MaxPooling2D((1,2), padding='same')(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
x = MaxPooling2D((1,2), padding='same')(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
x = MaxPooling2D((1,2), padding='same')(x)

encoded2 = Reshape((25,64))(x)



# Define decoder architecture
x = Reshape((25,1,64))(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
x = UpSampling2D((1,2))(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
x = UpSampling2D((1,2))(x)
x = Conv2D(128, 3, activation='relu', padding='same')(x)
x = UpSampling2D((1,2))(x)
x=Conv2D(216, 3, activation='sigmoid', padding='same')(x)
x=Reshape((25,6,288))(x)

decoded2 = Conv2D(848, 3, activation='sigmoid', padding='same')(x)

# Define autoencoder model
autoencoder2 = Model(inputs=inputs, outputs=decoded2)

# Compile model
autoencoder2.compile(optimizer='adam', loss='mse')

# Print model summary
print(autoencoder2.summary())


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 25, 6, 848)]      0         
                                                                 
 conv2d (Conv2D)             (None, 25, 6, 128)        977024    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 3, 128)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 3, 64)         73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 2, 64)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 2, 64)         3692

In [11]:
# Train the model
autoencoder2.fit(X1, X1,
                epochs=10,
                batch_size=64,
                shuffle=True,
                validation_data=(X2, X2)
               )

Epoch 1/10
433/433 [==============================] - 237s 546ms/step - loss: 0.0450 - val_loss: 0.0518
Epoch 2/10
433/433 [==============================] - 234s 541ms/step - loss: 0.0425 - val_loss: 0.0518
Epoch 3/10
433/433 [==============================] - 236s 546ms/step - loss: 0.0425 - val_loss: 0.0520
Epoch 4/10
433/433 [==============================] - 235s 544ms/step - loss: 0.0426 - val_loss: 0.0516
Epoch 5/10
433/433 [==============================] - 238s 549ms/step - loss: 0.0425 - val_loss: 0.0518
Epoch 6/10
433/433 [==============================] - 236s 546ms/step - loss: 0.0426 - val_loss: 0.0523
Epoch 7/10
433/433 [==============================] - 240s 554ms/step - loss: 0.0425 - val_loss: 0.0517
Epoch 8/10
433/433 [==============================] - 235s 544ms/step - loss: 0.0426 - val_loss: 0.0519
Epoch 9/10
433/433 [==============================] - 235s 543ms/step - loss: 0.0426 - val_loss: 0.0518
Epoch 10/10
433/433 [==============================] - 236s 544m

In [12]:
encoder_2 = Model(inputs, encoded2)
encoder_2.compile(optimizer='adam', loss='mse')

In [13]:
encoder_2.save("encoder_cnn_25.h5")